In [68]:
import csv
import random
import numpy as np
import pandas as pd
import dask.dataframe as dd
import os
from tqdm import tqdm

data = pd.read_csv('D:\\Projects\Data\\train.csv',nrows = 10000000)       # Contains the first 10 million data from 
                                                    # where we are going to select 1 million ramdom data for training and
                                                    # ten thousand data for deveopment 


In [69]:
# Creation of training set , development set and test set 
training_set = data.sample(n = 1000000)    # 1 million random data from same distribution
dev_set = data.sample(n = 10000)           # 10000 random data from same distribution
test_set =  pd.read_csv('D:\\Projects\\Data\\test.csv')      # Actual test data must be used to check only after getting 
                                                             # a good final hypothesis

# Shape, data type, data sample and descriptive statistics of Training Set

### Shape

In [70]:
training_set.shape

(1000000, 8)

### Data Type

In [71]:
training_set.dtypes

key                   object
fare_amount          float64
pickup_datetime       object
pickup_longitude     float64
pickup_latitude      float64
dropoff_longitude    float64
dropoff_latitude     float64
passenger_count        int64
dtype: object

### Data Sample

In [72]:
training_set.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
5637465,2012-11-10 15:14:00.00000058,10.50,2012-11-10 15:14:00 UTC,-73.961972,40.770672,-73.961072,40.761670,1
8152448,2014-11-02 19:02:00.000000160,30.83,2014-11-02 19:02:00 UTC,-73.873010,40.774160,-73.952800,40.773980,1
5810380,2014-01-23 12:31:29.0000004,4.50,2014-01-23 12:31:29 UTC,-74.013073,40.716552,-74.008597,40.716010,1
5585785,2014-03-14 00:03:00.000000116,28.83,2014-03-14 00:03:00 UTC,-73.873200,40.774155,-73.952982,40.768987,1
4154405,2009-06-13 06:28:00.0000009,7.30,2009-06-13 06:28:00 UTC,-73.948008,40.778548,-73.968808,40.791138,1


### Descriptive statistics 

In [73]:
training_set.describe()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,1000000.000000,1000000.000000,1000000.000000,999996.000000,999996.000000,1000000.000000
mean,11.341818,-72.512398,39.907714,-72.505052,39.915016,1.686525
std,9.797023,12.850012,9.245128,12.954164,9.776248,1.316056
min,-107.750000,-3313.387292,-3458.664702,-3426.600617,-2545.761383,0.000000
25%,6.000000,-73.992111,40.734860,-73.991405,40.733978,1.000000
50%,8.500000,-73.981835,40.752607,-73.980164,40.753120,1.000000
75%,12.500000,-73.967138,40.767048,-73.963680,40.768105,2.000000
max,477.920000,3061.872988,2925.069102,3061.872988,3351.403027,129.000000


# Shape, data type, data sample and descriptive statistics of Development Set

### Shape

In [74]:
dev_set.shape

(10000, 8)

### Data Type

In [75]:
dev_set.dtypes

key                   object
fare_amount          float64
pickup_datetime       object
pickup_longitude     float64
pickup_latitude      float64
dropoff_longitude    float64
dropoff_latitude     float64
passenger_count        int64
dtype: object

### Data Sample

In [76]:
dev_set.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
2634480,2014-07-22 13:22:48.0000001,7.0,2014-07-22 13:22:48 UTC,-73.983290,40.693928,-73.995587,40.716787,1
940510,2013-11-09 18:06:37.0000001,6.5,2013-11-09 18:06:37 UTC,-74.002454,40.745166,-74.004638,40.730722,1
7985928,2012-02-25 18:06:26.0000002,7.3,2012-02-25 18:06:26 UTC,-73.985640,40.722129,-74.005338,40.724472,1
5252800,2013-01-01 19:49:00.00000059,4.5,2013-01-01 19:49:00 UTC,-73.992917,40.730870,-73.987465,40.732972,1
6581147,2011-12-02 00:29:58.0000004,5.7,2011-12-02 00:29:58 UTC,-73.986625,40.733178,-73.975940,40.751380,1


### Descriptive statistics 

In [77]:
dev_set.describe()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,11.343942,-72.358182,39.885921,-72.349693,39.845777,1.695900
std,9.579253,10.878636,7.225359,10.907602,6.219125,1.331766
min,2.500000,-74.232109,-74.000797,-74.616665,-74.009980,0.000000
25%,6.000000,-73.991821,40.735254,-73.991432,40.733406,1.000000
50%,8.500000,-73.981695,40.752282,-73.980418,40.753393,1.000000
75%,12.900000,-73.966674,40.766868,-73.963774,40.768084,2.000000
max,140.000000,40.762522,410.333332,40.774315,48.866667,6.000000


# Shape, data type, data sample and descriptive statistics of Test Set



### Shape

In [78]:
test_set.shape

(9914, 7)

### Data Type

In [79]:
test_set.dtypes

key                   object
pickup_datetime       object
pickup_longitude     float64
pickup_latitude      float64
dropoff_longitude    float64
dropoff_latitude     float64
passenger_count        int64
dtype: object

### Data Sample

In [80]:
test_set.head()

,key,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2015-01-27 13:08:24.0000002,2015-01-27 13:08:24 UTC,-73.973320,40.763805,-73.981430,40.743835,1
1,2015-01-27 13:08:24.0000003,2015-01-27 13:08:24 UTC,-73.986862,40.719383,-73.998886,40.739201,1
2,2011-10-08 11:53:44.0000002,2011-10-08 11:53:44 UTC,-73.982524,40.751260,-73.979654,40.746139,1
3,2012-12-01 21:12:12.0000002,2012-12-01 21:12:12 UTC,-73.981160,40.767807,-73.990448,40.751635,1
4,2012-12-01 21:12:12.0000003,2012-12-01 21:12:12 UTC,-73.966046,40.789775,-73.988565,40.744427,1


### Descriptive statistics 

In [81]:
test_set.describe()

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,9914.000000,9914.000000,9914.000000,9914.000000,9914.000000
mean,-73.974722,40.751041,-73.973657,40.751743,1.671273
std,0.042774,0.033541,0.039072,0.035435,1.278747
min,-74.252193,40.573143,-74.263242,40.568973,1.000000
25%,-73.992501,40.736125,-73.991247,40.735254,1.000000
50%,-73.982326,40.753051,-73.980015,40.754065,1.000000
75%,-73.968013,40.767113,-73.964059,40.768757,2.000000
max,-72.986532,41.709555,-72.990963,41.696683,6.000000


# Data Cleansing

## Removing Null values from the datasets

In [82]:
training_set.isnull().sum().sort_values(ascending = False)

dropoff_latitude     4
dropoff_longitude    4
passenger_count      0
pickup_latitude      0
pickup_longitude     0
pickup_datetime      0
fare_amount          0
key                  0
dtype: int64

In [83]:
dev_set.isnull().sum().sort_values(ascending = False)

passenger_count      0
dropoff_latitude     0
dropoff_longitude    0
pickup_latitude      0
pickup_longitude     0
pickup_datetime      0
fare_amount          0
key                  0
dtype: int64

In [84]:
test_set.isnull().sum().sort_values(ascending = False)

passenger_count      0
dropoff_latitude     0
dropoff_longitude    0
pickup_latitude      0
pickup_longitude     0
pickup_datetime      0
key                  0
dtype: int64

In [85]:
training_set = training_set.drop(training_set[training_set.isnull().any(1)].index , axis = 0)
training_set.shape

(999996, 8)

In [86]:
dev_set = dev_set.drop(dev_set[dev_set.isnull().any(1)].index , axis = 0)
dev_set.shape

(10000, 8)

In [87]:
test_set = test_set.drop(test_set[test_set.isnull().any(1)].index , axis = 0)
test_set.shape

(9914, 7)

## Removing rows from the dataset having invalid data 

In [88]:
training_set.describe()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,999996.000000,999996.000000,999996.000000,999996.000000,999996.000000,999996.000000
mean,11.341794,-72.512392,39.907710,-72.505052,39.915016,1.686532
std,9.797028,12.850038,9.245146,12.954164,9.776248,1.316055
min,-107.750000,-3313.387292,-3458.664702,-3426.600617,-2545.761383,0.000000
25%,6.000000,-73.992111,40.734860,-73.991405,40.733978,1.000000
50%,8.500000,-73.981835,40.752607,-73.980164,40.753120,1.000000
75%,12.500000,-73.967138,40.767048,-73.963680,40.768105,2.000000
max,477.920000,3061.872988,2925.069102,3061.872988,3351.403027,129.000000


In [89]:
training_set['fare_amount'].describe()

count    999996.000000
mean         11.341794
std           9.797028
min        -107.750000
25%           6.000000
50%           8.500000
75%          12.500000
max         477.920000
Name: fare_amount, dtype: float64

### Training_set is having negative value fare so we will remove those rows

In [90]:
from collections import Counter
Counter(training_set['fare_amount']<0)

Counter({False: 999946, True: 50})

In [91]:
training_set = training_set.drop(training_set[training_set['fare_amount']<0].index, axis=0)
training_set.shape

(999946, 8)

In [92]:
training_set['fare_amount'].describe()

count    999946.000000
mean         11.342850
std           9.795259
min           0.000000
25%           6.000000
50%           8.500000
75%          12.500000
max         477.920000
Name: fare_amount, dtype: float64

### Passenger count is greater than  7

In [93]:
training_set['passenger_count'].describe()

count    999946.000000
mean          1.686520
std           1.316036
min           0.000000
25%           1.000000
50%           1.000000
75%           2.000000
max         129.000000
Name: passenger_count, dtype: float64

In [94]:
#len(training_set[training_set['passenger_count']>7])

In [95]:
#### Since we are restricting to yellow cabs or medallion taxi. So we are considering 
#### a maximum of 7 passengers can operate in a taxi at a time.
Counter(training_set['passenger_count']>8)

Counter({False: 999945, True: 1})

In [96]:
training_set = training_set.drop(training_set[training_set['passenger_count']>8].index, axis = 0)
training_set.describe()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,999945.000000,999945.000000,999945.000000,999945.000000,999945.000000,999945.000000
mean,11.342852,-72.512688,39.907871,-72.505421,39.915218,1.686393
std,9.795263,12.849339,9.244949,12.953273,9.776007,1.309864
min,0.000000,-3313.387292,-3458.664702,-3426.600617,-2545.761383,0.000000
25%,6.000000,-73.992111,40.734860,-73.991405,40.733979,1.000000
50%,8.500000,-73.981836,40.752607,-73.980164,40.753120,1.000000
75%,12.500000,-73.967139,40.767048,-73.963682,40.768105,2.000000
max,477.920000,3061.872988,2925.069102,3061.872988,3351.403027,7.000000


### Removing invalid Longitude and Latitude 

In [97]:
training_set[['pickup_longitude','pickup_latitude']].describe()

,pickup_longitude,pickup_latitude
count,999945.000000,999945.000000
mean,-72.512688,39.907871
std,12.849339,9.244949
min,-3313.387292,-3458.664702
25%,-73.992111,40.734860
50%,-73.981836,40.752607
75%,-73.967139,40.767048
max,3061.872988,2925.069102


In [98]:
training_set[['dropoff_longitude','dropoff_latitude']].describe()

,dropoff_longitude,dropoff_latitude
count,999945.000000,999945.000000
mean,-72.505421,39.915218
std,12.953273,9.776007
min,-3426.600617,-2545.761383
25%,-73.991405,40.733979
50%,-73.980164,40.753120
75%,-73.963682,40.768105
max,3061.872988,3351.403027


#### Valid Longitude value = -180 to 180
#### Valid Latitude value = -90 to 90
#### Remove pickup cordinates and drop off cordinates which dont fall in this range 

##### Remove invalid pickup cordinates 

In [99]:
print("Number of invalid pickup cordinates = ",len((training_set[training_set['pickup_latitude']<-90])|(training_set[training_set['pickup_latitude']>90])|(training_set[training_set['pickup_longitude']<-180])|(training_set[training_set['pickup_longitude']>180])))

Number of invalid pickup cordinates =  30


In [100]:
training_set = training_set.drop(((training_set[training_set['pickup_latitude']<-90])|(training_set[training_set['pickup_latitude']>90])|(training_set[training_set['pickup_longitude']<-180])|(training_set[training_set['pickup_longitude']>180])).index, axis=0)
training_set.shape

(999915, 8)

##### Remove invalid drop off cordinates  

In [101]:
print("Number of invalid drop off cordinates = ",len((training_set[training_set['dropoff_latitude']<-90])|(training_set[training_set['dropoff_latitude']>90])|(training_set[training_set['dropoff_longitude']<-180])|(training_set[training_set['dropoff_longitude']>180])))

Number of invalid drop off cordinates =  21


In [102]:
training_set = training_set.drop(((training_set[training_set['dropoff_latitude']<-90])|(training_set[training_set['dropoff_latitude']>90])|(training_set[training_set['dropoff_longitude']<-180])|(training_set[training_set['dropoff_longitude']>180])).index, axis=0)
training_set.shape

(999894, 8)

## Distance between pick up location and drop off location
#### The distance in a sphere can be calculated when latitudes and longitudes are given by Haversine formula
#### haversine(θ) = sin²(θ/2)

#### φ is latitude, λ is longitude, R is earth’s radius (mean radius = 6,371km) to include latitude and longitude coordinates (A and B in this case).

#### a = sin²((φB - φA)/2) + cos φA . cos φB . sin²((λB - λA)/2)

#### c = 2 * atan2( √a, √(1−a) )

#### d = R ⋅ c

#### d = Haversine distance

In [103]:
def haversine_distance(data):
    R = 6371        # Radius of earth in Kilo Meter
    
    pickup_data = data[['pickup_longitude','pickup_latitude']]
    dropoff_data = data[['dropoff_longitude','dropoff_latitude']]
    
    phi1 = np.radians(pickup_data['pickup_latitude'])
    phi2 = np.radians(dropoff_data['dropoff_latitude'])
    
    delta_phi = np.radians(dropoff_data['dropoff_latitude'] - pickup_data['pickup_latitude'])
    delta_lambda = np.radians(dropoff_data['dropoff_longitude'] - pickup_data['pickup_longitude'])
    
    #a = sin²((φB - φA)/2) + cos φA . cos φB . sin²((λB - λA)/2)
    a = np.sin((delta_phi)/2)**2 + np.cos(phi1)*np.cos(phi2)*np.sin(delta_lambda/2)**2
    
    #c = 2 * atan2( √a, √(1−a) )
    c = 2 * np.arctan2(np.sqrt(a),np.sqrt(1-a))
    d = R * c
    
    return d
    

In [108]:
training_set_distance = pd.DataFrame(haversine_distance(training_set) , columns=['Distance'])
training_set = pd.concat([training_set , training_set_distance] , axis = 1)

In [106]:
training_set.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,Distance
5637465,2012-11-10 15:14:00.00000058,10.50,2012-11-10 15:14:00 UTC,-73.961972,40.770672,-73.961072,40.761670,1,1.003842
8152448,2014-11-02 19:02:00.000000160,30.83,2014-11-02 19:02:00 UTC,-73.873010,40.774160,-73.952800,40.773980,1,6.718897
5810380,2014-01-23 12:31:29.0000004,4.50,2014-01-23 12:31:29 UTC,-74.013073,40.716552,-74.008597,40.716010,1,0.382022
5585785,2014-03-14 00:03:00.000000116,28.83,2014-03-14 00:03:00 UTC,-73.873200,40.774155,-73.952982,40.768987,1,6.742977
4154405,2009-06-13 06:28:00.0000009,7.30,2009-06-13 06:28:00 UTC,-73.948008,40.778548,-73.968808,40.791138,1,2.242011


In [107]:
training_set.describe()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,Distance
count,999894.000000,999894.000000,999894.000000,999894.000000,999894.000000,999894.000000,999894.000000
mean,11.342862,-72.498237,39.915428,-72.494991,39.914504,1.686376,19.585111
std,9.795322,10.460383,6.133137,10.469362,6.136496,1.309844,368.481933
min,0.000000,-121.914810,-74.016942,-173.342034,-74.031022,0.000000,0.000000
25%,6.000000,-73.992111,40.734861,-73.991405,40.733980,1.000000,1.214607
50%,8.500000,-73.981835,40.752607,-73.980164,40.753120,1.000000,2.114807
75%,12.500000,-73.967139,40.767048,-73.963681,40.768105,2.000000,3.875985
max,477.920000,40.850357,81.545448,45.581619,81.545448,7.000000,12594.704725
